In [5]:
init_dict = {'age_years': 0,
            'relationship_to_primary_beneficiary': 0,
            'clinic_visits': 0,
            'clinic_inpatient': 0,
            'region_mod_northcentral': 0,
            'region_mod_northeast': 0,
            'region_mod_south': 0,
            'region_mod_west': 0,
            'gender_female':0,
            'gender_male':0,
            'both_clinic':0,
            'cci': 0}
init_lst = list(init_dict.keys())

disease_name_mapping = {
        'respiratory_d': 'Respiratory Disease',
        'hypertension': 'Hypertension',
        'diabetes_melitus': 'Diabetes Melitus',
        'dementia': 'Dementia',
        'kidney_disease': 'Kidney Disease',
        'liver_disease': 'Liver Disease',
        'diarrheal_disease': 'Diarrheal Disease',
        'myocardial_infarction': 'Myocardial Infarction',
        'cardiovascular_d': 'Cardiovascular Disease',
        'chf': 'Heart Failure',
        'pvd': 'Peripherial Vascular Disease',
        'cancer': 'Non-Metastatic Cancer',
        'metastasis': 'Metastatic Cancer',
        'connective_tissue_disease': 'Autoimmune Disease',
        'puc': 'Peptic Ulcer',
        'hemiplegia': 'Stroke',
        'lymphoma': 'Lymphoma',
        'aids': 'AIDS',
        'trauma': 'Previous Fracture'
    }
init_lst.extend(list(disease_name_mapping.keys()))
column_order =init_lst

In [7]:
init_lst

['age_years',
 'relationship_to_primary_beneficiary',
 'clinic_visits',
 'clinic_inpatient',
 'region_mod_northcentral',
 'region_mod_northeast',
 'region_mod_south',
 'region_mod_west',
 'gender_female',
 'gender_male',
 'both_clinic',
 'cci',
 'respiratory_d',
 'hypertension',
 'diabetes_melitus',
 'dementia',
 'kidney_disease',
 'liver_disease',
 'diarrheal_disease',
 'myocardial_infarction',
 'cardiovascular_d',
 'chf',
 'pvd',
 'cancer',
 'metastasis',
 'connective_tissue_disease',
 'puc',
 'hemiplegia',
 'lymphoma',
 'aids',
 'trauma']

In [18]:
# Standard library imports
import math
import os

# Related third party imports
import janitor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from dotenv import load_dotenv
from google.cloud import bigquery
from sklearn.inspection import permutation_importance
from sklearn.linear_model import Ridge, SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, max_error
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LSTM, LeakyReLU
import pydoc_data
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
# Local application/library specific imports
# (None in this example, but this is where they would go)
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
# Load the environment variables from the .env file
load_dotenv()


True

In [82]:

google_credentials = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
PROJECT = "focus-hulling-416322"
DATASET = "2016"
TABLE = "out_and_in_2016_v4"

# Ensure the project, dataset, and table names are correctly formatted
# using backticks to avoid syntax errors
query = f"""
    SELECT *
    FROM `{PROJECT}.{DATASET}.{TABLE}`
"""

client = bigquery.Client(project=PROJECT)
query_job = client.query(query)
result = query_job.result()
data = result.to_dataframe()


In [83]:
data.dtypes

Patient_ID                             float64
Gender                                  object
age_years                              float64
region_mod                              object
Relationship_To_Primary_Beneficiary     object
Patient_Zipcode                        float64
Health_Plan_Type                        object
plan_typ                               float64
Clinic                                   Int64
drg                                      Int64
Clinic_visits                            Int64
Myocardial_infarction                  float64
chf                                    float64
pvd                                    float64
Cardiovascular_d                       float64
Respiratory_d                          float64
Hypertension                           float64
Diabetes_Melitus                       float64
Dementia                               float64
Kidney_disease                         float64
Liver_disease                          float64
Diarrheal_dis

In [95]:
# Cleaning the column names
df = data.clean_names()

# Changin the values inside the clinic column for onehot encoding
value_map = {1: 'Inpatient', 2: 'Outpatient'}
df['clinic'] = df['clinic'].map(value_map)

#Chaning the values for region_mod
value_map_2 = {'1': 'northeast', '2':'northcentral', '3':'south', '4':'west', '5':'unknown'}
df['region_mod'] = df['region_mod'].map(value_map_2)

#chaning the gender varible
value_map_3 = {'1':'male', '2':'female'}
df['gender'] = df['gender'].map(value_map_3)

# # Creating values for Plantype
# value_map_4 = {1: 'basic', 2: 'comprihensive', 3:'epo', 4:'hmo', 5:'pos', 6:'ppo', 7:'pos2', 8:'cdhp', 9:'hdhp'}


def encode_and_bind(original_dataframe, features_to_encode):

    # Convert a single feature name to a list
    if isinstance(features_to_encode, str):
        features_to_encode = [features_to_encode]

    # Check if the features exist in the dataframe
    for feature in features_to_encode:
        if feature not in original_dataframe.columns:
            raise ValueError(f"Feature '{feature}' not found in the dataframe")

    # One-hot encoding
    encoder = OneHotEncoder(sparse_output = False)
    encoder.fit(original_dataframe[features_to_encode])
    encoded_features = encoder.transform(original_dataframe[features_to_encode])

    # Add new encoded columns to the dataframe
    original_dataframe[encoder.get_feature_names_out(features_to_encode)] = encoded_features

    # Drop the original columns
    original_dataframe.drop(columns=features_to_encode, inplace=True)

    # Return the modified dataframe
    return original_dataframe

features_to_encode = ['clinic', 'region_mod', 'gender']

encode_and_bind(df, features_to_encode)

value_to_encode = [183, 184, 185, 521, 522, 533, 534, 545, 536, 542, 543, 544, 562, 563]
df['trauma'] = df['drg'].apply(lambda x: 1 if x in value_to_encode else 0)
df.loc[df['drg'].between(280, 283), 'myocardial_infarction'] += 1
df = df.drop(columns=['health_plan_type','plan_typ', 'drg', 'patient_zipcode'])
# repaet clean_names()
# df = df.clean_names()

# group_columns = ['patient_id', 'gender_male','relationship_to_primary_beneficiary']
agg_dict = {col: 'sum' for col in df.columns if col not in group_columns}
agg_dict['age_years'] = 'mean'
df = df.groupby(group_columns).agg(agg_dict).reset_index()

# #sorting out patient_id column
df['patient_id'] = df['patient_id'].astype(str)
df['patient_id'] = df['patient_id'].str.slice(0, -2)

# #eliminating the negative pay values
df = df[(df['age_years'] > 17) & (df['pay'].between(2,1000000))]

df = df.clean_names()
# #removing the duplicates
duplicates = df.duplicated(subset='patient_id', keep=False)
duplicate_rows= df[duplicates]
duplicate_rows.sort_values(by='patient_id')
df.drop_duplicates(inplace=True)
df['both_clinic'] = np.where((df['clinic_inpatient'] == 1.0) & (df['clinic_outpatient'] == 1.0), 1,0)


df = df.astype(float)
columns_to_replace = [
    'myocardial_infarction', 'trauma', 'chf', 'pvd', 'cardiovascular_d'
    'respiratory_d', 'hypertension', 'diabetes_melitus', 'dementia',
    'kidney_disease', 'liver_disease', 'diarrheal_disease', 'cancer',
    'metastasis', 'puc', 'hemiplegia', 'lymphoma', 'aids',
    'connective_tissue_disease', 'region_mod_northcentral',
    'region_mod_northeast', 'region_mod_south', 'region_mod_west',
    'region_mod_unknown', 'clinic_outpatient', 'clinic_inpatient', 'both_clinic', 'gender_female',
    'gender_male'
]
for column in columns_to_replace:
    if column in df.columns:
        # Create a mask for values greater than zero
        mask = df[column] > 0
        # Apply the mask and set those values to 1
        df.loc[mask, column] = 1


# CCI score
df['cci'] = 0
df['cci']= pd.cut(df['age_years'],
                    bins=[0, 49, 59, 69, 79, float('inf')],
                    labels=[0, 1, 2, 3, 4],
                    right=False).astype(int)
df.loc[df['myocardial_infarction'] == 1.0, 'cci'] +=1
df.loc[df['chf'] == 1.0, 'cci'] +=1
df.loc[df['pvd'] == 1.0, 'cci'] +=1
df.loc[df['respiratory_d'] == 1.0, 'cci'] +=1
df.loc[df['connective_tissue_disease'] == 1.0, 'cci'] +=1
df.loc[df['liver_disease'] == 1.0, 'cci'] +=1
df.loc[df['diabetes_melitus'] == 1.0, 'cci'] +=1
df.loc[df['kidney_disease'] == 1.0, 'cci'] +=1
df.loc[df['puc'] == 1.0, 'cci'] +=1
df.loc[df['hemiplegia'] == 1.0, 'cci'] +=2
df.loc[df['lymphoma'] == 1.0, 'cci'] +=2
df.loc[df['aids'] == 1.0, 'cci'] +=6
df.loc[df['cancer'] == 1.0, 'cci'] +=2
df.loc[df['metastasis'] == 1.0, 'cci'] +=6

scaling_columns = ['lohs', 'clinic_visits', 'age_years']
def scaler_minmax(data_frame, scaling_columns):
    minmax = MinMaxScaler()
    if isinstance(scaling_columns, str):  # If a single column name is passed as a string
        scaling_columns = [scaling_columns]  # Convert it to a list

    minmax.fit(data_frame[scaling_columns])
    data_frame[scaling_columns] = minmax.transform(data_frame[scaling_columns])
scaler_minmax(df, scaling_columns)

## relationship to primary beneficiary is to high, the numbers have been added up
## health plan type

,patient_id,age_years,relationship_to_primary_beneficiary,clinic_visits,myocardial_infarction,chf,pvd,cardiovascular_d,respiratory_d,hypertension,...,region_mod_northcentral,region_mod_northeast,region_mod_south,region_mod_unknown,region_mod_west,gender_female,gender_male,trauma,both_clinic,cci
0,2.030200e+04,0.914894,2.0,0.014272,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2
1,5.860200e+04,0.978723,2.0,0.001903,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2
2,6.820200e+04,0.744681,33.0,0.074215,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2
3,1.476010e+05,0.914894,33.0,0.059943,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3
4,2.026030e+05,0.893617,2.0,0.050428,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2298456,3.404381e+10,0.872340,1.0,0.006660,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3
2298457,3.404382e+10,0.297872,1.0,0.006660,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2298458,3.404382e+10,0.723404,2.0,0.028544,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
2298459,3.404382e+10,0.382979,1.0,0.003806,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


### outliers ###

## Preparing the X and the y

In [109]:
# Preparing the data

X = df.drop(columns=['pay', 'clinic_outpatient','patient_zip' 'patient_id', 'coinsurance', 'copay', 'deductable', 'net', 'region_mod_unknown'])

y = df['pay']


# Gradient Boosting Machine

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [111]:
gbm = cross_validate(GradientBoostingRegressor(n_estimators=200,
                                               learning_rate=0.1,
                                               min_samples_leaf=7,
                                               max_depth=8,
                                               min_samples_split=30,
                                               random_state=42),
                     X_train,
                     y_train,
                     cv=5,
                     scoring='r2')

test_score_gbm = gbm['test_score'].mean()
test_score_gbm

0.6253073655566777

In [122]:
gbm = GradientBoostingRegressor(n_estimators=200,
                                min_samples_split=30,
                                min_samples_leaf=7,
                                max_depth=8 ,
                                learning_rate=0.1,
                                random_state=42)
gbm.fit(X_train, y_train)

y_pred = gbm.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Squared Error: {mae}")
#670160468.497997

Mean Squared Error: 662402098.5054079
